# Normalize sqY2H consensus scores

This script applies various QA/QC steps and then computes a normalized interaction score. For QA/QC we:

1.   Check whether 6 media controls on each mega-plate meet various condition-specific thresholds
2.   Check for autoactivation mating failures
3.   Adjust consensus score based on autoactivation score and filter if score invalid

In [1]:
import polars as pl 
import pandas as pd
import numpy as np

In [2]:
inputs_dir = "../1_inputs/sqY2H"
outputs_dir = "../3_outputs/sqY2H"
meta_inputs = "../../../1_allele_collection/1_inputs"
meta_outputs = "../../../1_allele_collection/3_outputs"

scoring_conditions = ['LWH1', 'LWH10', 'LWH25', 'LWA']
max_score = 4
aa_id = '0.0'
rrs_id = 999999.0

In [3]:
def df_to_dict_variants(df):
    if isinstance(df, pl.DataFrame):
        df = df.to_pandas()
        
    if 'media_control' in df.columns:
        df = df[df['media_control'] == False].copy()
        
    scoring_conditions = ['LWH1', 'LWH10', 'LWH25', 'LWA']
        
    if 'condition_name' in df.columns:
        df = df[df['condition_name'].isin(scoring_conditions)].copy()
    
    cols_to_keep = ['db_orf_id', 'ad_orf_id', 'db_mut_id', 'retest_batch']

    df = df[cols_to_keep].copy()

    df[['db_orf_id', 'ad_orf_id', 'db_mut_id']] = df[['db_orf_id', 'ad_orf_id', 'db_mut_id']].map(float).map(int)
        
    def filter_df(df):
        df = df[(df.ad_orf_id != 0) 
                & (df.db_mut_id != 999999)].copy()
        
        return df
    
    df = filter_df(df)
    
    df_dict = {}
    
    assay_array = ["VUSAPWT1B1","VUSAPWT1B2","VUSAPWT2B1","VUSAPWT6B1"]
    
    for retest_batch in assay_array:
        df_assay = df[df['retest_batch'] == retest_batch]
        df_dict[retest_batch] = {}
        for db_orf_id, df_db in df_assay.sort_values('db_orf_id').groupby('db_orf_id'):
            df_dict[retest_batch][db_orf_id] = {}
            for i, row in df_db.sort_values('db_mut_id').iterrows():
                db_mut_id = row['db_mut_id']
                ad_orf_id = row['ad_orf_id']
                if db_mut_id not in df_dict[retest_batch][db_orf_id].keys():
                    df_dict[retest_batch][db_orf_id][db_mut_id] = {ad_orf_id}
                else:
                    df_dict[retest_batch][db_orf_id][db_mut_id].add(ad_orf_id)
                    
    return df_dict

## Apply all QA/QC thresholds

In [4]:
# get media control thresholds
media_score_thresholds = pd.read_csv(f"{inputs_dir}/media_control_score_thresholds.csv")
media_score_thresholds.rename(columns={'control_96': 'scoring_pos'}, inplace=True)

In [5]:
scores = pd.read_csv(f"{outputs_dir}/2_raw_consensus_scores.csv") 

# Split into media controls
media_ctrls = scores[scores['media_control']]
scores = scores[~scores['media_control']]

# Check media controls
media_ctrls = media_ctrls[['db_orf_id', 'condition_name', 'retest_batch', 'consensus_score']]
media_ctrls[['scoring_pos', 'retest_pla']] = media_ctrls['db_orf_id'].str.split('_', n=1, expand=True)
media_ctrls = media_ctrls.merge(media_score_thresholds, on=['scoring_pos', 'condition_name'])
media_ctrls['score_in_bounds'] = media_ctrls['consensus_score'].between(media_ctrls['lower_bound'], media_ctrls['upper_bound'], inclusive='both')

# replace failure due to variable scoring (99999) with NaN to group with other technical failures
scores['consensus_score'] = scores['consensus_score'].replace(99999, np.nan)

# add IDs for groups used for filtering
scores = pl.DataFrame(scores)

scores = scores.with_columns(
    pl.col("retest_batch").str.extract(r"VUSAPWT(\d)B\d+", 1).alias("assay_id")
)

scores = scores.with_columns(
    pl.concat_str([
            pl.col('db_orf_id').cast(pl.String),
            pl.col('db_mut_id').cast(pl.String)
    ], separator="_").alias("db_group"),
    pl.concat_str([
            pl.col('db_orf_id').cast(pl.String),
            pl.col('db_mut_id').cast(pl.String),
            pl.col('condition_name').cast(pl.String),
            pl.col('retest_batch').cast(pl.String)
    ], separator="_").alias("db_condition_group"),
    pl.concat_str([
            pl.col('ad_orf_id').cast(pl.String),
            pl.col('db_orf_id').cast(pl.String),
            pl.col('db_mut_id').cast(pl.String),
            pl.col('retest_batch').cast(pl.String)
    ], separator="_").alias("ppi_group")
)

dict_before_qc = df_to_dict_variants(scores)

In [6]:
# Remove all db groups with the same allele if there was an autoactivation mating failure
aa_db_group_mating_failures = (scores.filter(pl.col("ad_orf_id") == aa_id)
                              .filter(pl.col("condition_name") == "LW")
                              .filter(pl.col("consensus_score") <= 2)
                              .select("db_group")
                              .to_series().unique().to_list())
scores = scores.filter(~pl.col("db_group").is_in(aa_db_group_mating_failures))

dict_after_db_groups = df_to_dict_variants(scores)

# Remove all db-condition groups with the same allele if there was an autoactivation mating failure
aa_db_group_tech_failures = (scores.filter(pl.col("ad_orf_id") == aa_id)
                                   .filter(pl.col("consensus_score").is_nan())
                                   .select("db_condition_group")
                                   .to_series().unique().to_list())
scores = scores.filter(~pl.col("db_condition_group").is_in(aa_db_group_tech_failures))

dict_after_db_conditions = df_to_dict_variants(scores)

# remove mating failures - condition 1 shows no or little yeast growth
mating_failures = (scores.filter(pl.col("condition_name") == "LW")
                         .filter(pl.col("consensus_score") <= 2)
                         .select("ppi_group").to_series().unique().to_list())
scores = scores.filter(~pl.col("ppi_group").is_in(mating_failures))

dict_after_mating = df_to_dict_variants(scores)


# Adjust consensus score based on autoactivation score
aa_scores = scores.filter(pl.col("ad_orf_id") == aa_id).select([
    "db_condition_group", "consensus_score"
]).rename({"consensus_score": "aa_score"})

# Remove autoactivation rows from the main scores df
# Add the aa_score to the dataframe, merged on the condition-specific db allele
scores = scores.filter(pl.col("ad_orf_id") != aa_id).join(aa_scores, on="db_condition_group")

scores = scores.with_columns(
    (pl.col("consensus_score") - pl.col("aa_score")).alias("adjusted_score")
).filter(
    ~((pl.col("adjusted_score") <= 0) & (pl.col("aa_score") > 0))
)

dict_after_aa = df_to_dict_variants(scores)

In [7]:
# split rrs & wt scores
rrs_scores = scores.filter(pl.col('db_mut_id') == rrs_id).filter(pl.col("condition_name") == "LWH1")
rrs_count_batch = rrs_scores.group_by(["retest_batch", "adjusted_score"]).agg(pl.len()).pivot(
    values="len", on="adjusted_score", index="retest_batch"
).select(["retest_batch", "0.0", "1.0", "2.0", "3.0", "4.0", "null"])
print(rrs_count_batch)

wt_scores = scores.filter(pl.col('db_mut_id') == 0.0).filter(pl.col("condition_name") == "LWH1")
wt_count_batch = wt_scores.group_by(["retest_batch", "adjusted_score"]).agg(pl.len()).pivot(
    values="len", on="adjusted_score", index="retest_batch"
).select(["retest_batch", "0.0", "1.0", "2.0", "3.0", "4.0", "null"])
print(wt_count_batch)

shape: (4, 7)
┌──────────────┬─────┬─────┬─────┬──────┬──────┬──────┐
│ retest_batch ┆ 0.0 ┆ 1.0 ┆ 2.0 ┆ 3.0  ┆ 4.0  ┆ null │
│ ---          ┆ --- ┆ --- ┆ --- ┆ ---  ┆ ---  ┆ ---  │
│ str          ┆ u32 ┆ u32 ┆ u32 ┆ u32  ┆ u32  ┆ u32  │
╞══════════════╪═════╪═════╪═════╪══════╪══════╪══════╡
│ VUSAPWT1B1   ┆ 153 ┆ 1   ┆ 3   ┆ 2    ┆ 2    ┆ null │
│ VUSAPWT1B2   ┆ 147 ┆ 4   ┆ 3   ┆ null ┆ null ┆ 3    │
│ VUSAPWT2B1   ┆ 165 ┆ 2   ┆ 3   ┆ 4    ┆ null ┆ null │
│ VUSAPWT6B1   ┆ 165 ┆ 2   ┆ 1   ┆ 1    ┆ 2    ┆ null │
└──────────────┴─────┴─────┴─────┴──────┴──────┴──────┘
shape: (4, 7)
┌──────────────┬─────┬─────┬─────┬─────┬─────┬──────┐
│ retest_batch ┆ 0.0 ┆ 1.0 ┆ 2.0 ┆ 3.0 ┆ 4.0 ┆ null │
│ ---          ┆ --- ┆ --- ┆ --- ┆ --- ┆ --- ┆ ---  │
│ str          ┆ u32 ┆ u32 ┆ u32 ┆ u32 ┆ u32 ┆ u32  │
╞══════════════╪═════╪═════╪═════╪═════╪═════╪══════╡
│ VUSAPWT6B1   ┆ 73  ┆ 6   ┆ 17  ┆ 127 ┆ 68  ┆ 2    │
│ VUSAPWT1B1   ┆ 3   ┆ 4   ┆ 13  ┆ 19  ┆ 65  ┆ null │
│ VUSAPWT1B2   ┆ 144 ┆ 19  ┆ 70  ┆

In [8]:
# Keep only scores that aren't RRS pairs
scores = scores.filter(pl.col('db_mut_id') != rrs_id)

scores = scores.with_columns(
    pl.col("retest_batch").str.extract(r"VUSAPWT(\d)B\d+", 1).alias("assay_id")
)

In [10]:
# read in sequence confirmation files
ad_seq = pl.read_csv(f"{inputs_dir}/swimseq_confirmation/vus_ad_interactors.csv").rename({
    "swim_seq_validated": "seq_ad"
}).select([
    "ad_orf_id", "assay_id", "seq_ad"
])

db_var_seq = pl.read_csv(f"{inputs_dir}/swimseq_confirmation/vus_db_alleles.csv").rename({
    "orf_id": "db_orf_id",
    "mut_id": "db_mut_id",
    "swim_seq_validated": "seq_db"
}).select(["db_orf_id", "db_mut_id", "seq_db"])

db_ref_seq = pl.read_csv(f"{inputs_dir}/swimseq_confirmation/wt_disease_nodes_summary.csv").rename({
    "disease_orf_id": "db_orf_id",
    "swim_seq_validated": "seq_db"
}).select(["db_orf_id", "seq_db"]).with_columns(
    pl.lit(0).cast(pl.Int64).alias("db_mut_id")
).select(["db_orf_id", "db_mut_id", "seq_db"])

db_seq = pl.concat([db_var_seq, db_ref_seq])
# print(db_seq)

## Add an extra QC step to check the sequence confirmation results
seq_confirm_res_df = pd.read_table(f"{meta_inputs}/seq_confirmation_results/VarChampSeqConfirmationResult.tsv")
# with pd.option_context('display.max_columns', None):
#     display(seq_confirm_res_df)

db_seq = pl.DataFrame(
    pd.merge(db_seq.to_pandas(), 
            seq_confirm_res_df[
                ["orf_id_wt","mutation_id_old","db_sequenced","db_sequence_confirmation_class"] ##"symbol","aa_change","nt_change","ccsb_mutation_id"
            ].rename({"orf_id_wt": "db_orf_id",
                      "mutation_id_old": "db_mut_id"}, axis=1), 
        on=["db_orf_id","db_mut_id"],
        how="left"
    )
)
print(db_seq)
# print(db_seq["db_sequenced"].unique())
# print(db_seq.filter(pl.col("db_sequenced")==0))

shape: (2_887, 5)
┌───────────┬───────────┬────────┬──────────────┬────────────────────────────────┐
│ db_orf_id ┆ db_mut_id ┆ seq_db ┆ db_sequenced ┆ db_sequence_confirmation_class │
│ ---       ┆ ---       ┆ ---    ┆ ---          ┆ ---                            │
│ i64       ┆ i64       ┆ str    ┆ f64          ┆ f64                            │
╞═══════════╪═══════════╪════════╪══════════════╪════════════════════════════════╡
│ 281       ┆ 727       ┆ NULL   ┆ 1.0          ┆ 1.0                            │
│ 281       ┆ 739       ┆ 1      ┆ 1.0          ┆ 1.0                            │
│ 281       ┆ 13        ┆ 1      ┆ 1.0          ┆ 1.0                            │
│ 281       ┆ 744       ┆ 1      ┆ 1.0          ┆ 1.0                            │
│ 281       ┆ 763       ┆ 1      ┆ 1.0          ┆ 1.0                            │
│ …         ┆ …         ┆ …      ┆ …            ┆ …                              │
│ 8514      ┆ 0         ┆ 1      ┆ null         ┆ null               

In [12]:
print(scores.shape)

(40034, 24)


In [ ]:
# cast all to same format
scores = scores.with_columns(
    pl.col("ad_orf_id").cast(pl.Float64).cast(pl.Int64).alias("ad_orf_id"),
    pl.col("db_orf_id").cast(pl.Float64).cast(pl.Int64).alias("db_orf_id"),
    pl.col("db_mut_id").cast(pl.Float64).cast(pl.Int64).alias("db_mut_id"),
    pl.col("assay_id").cast(pl.String).alias("assay_id"),
)

db_seq = db_seq.with_columns(
    pl.col("db_orf_id").cast(pl.Float64).cast(pl.Int64).alias("db_orf_id"),
    pl.col("db_mut_id").cast(pl.Float64).cast(pl.Int64).alias("db_mut_id"),
    pl.col("seq_db").cast(pl.String).alias("seq_db")
)

ad_seq = ad_seq.with_columns(
    pl.col("ad_orf_id").cast(pl.Float64).cast(pl.Int64).alias("ad_orf_id"),
    pl.col("seq_ad").cast(pl.String).alias("seq_ad"),
    pl.col("assay_id").cast(pl.String).alias("assay_id"),
)

scores = scores.join(ad_seq, on=["ad_orf_id", "assay_id"], how="left")
scores = scores.join(db_seq, on=["db_orf_id", "db_mut_id"], how="left")

# print(scores.filter(pl.col("db_mut_id")==0))

## Filter to only keep interactions where both ad and db are confirmed
## And also confirmed by the sequence confirmation results
valid_db_sequence_confirmation_class = [1., 2., 7., 99.]
scores = scores.filter(
    (pl.col("seq_ad") == "1") & (pl.col("seq_db") == "1") & (((pl.col("db_sequence_confirmation_class").is_in(valid_db_sequence_confirmation_class)) & (pl.col("db_sequenced").is_not_null())) | (pl.col("db_mut_id")==0))
)

## After sequence confirmation, we lost 25521 - 21895 = 3626 interactions
# print(scores.shape)
print(scores)

shape: (21_895, 28)
┌───────────┬───────────┬───────────┬───────────┬───┬────────┬────────┬──────────────┬─────────────┐
│ ad_orf_id ┆ ad_mut_id ┆ db_orf_id ┆ db_mut_id ┆ … ┆ seq_ad ┆ seq_db ┆ db_sequenced ┆ db_sequence │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---    ┆ ---    ┆ ---          ┆ _confirmati │
│ i64       ┆ f64       ┆ i64       ┆ i64       ┆   ┆ str    ┆ str    ┆ f64          ┆ on_class    │
│           ┆           ┆           ┆           ┆   ┆        ┆        ┆              ┆ ---         │
│           ┆           ┆           ┆           ┆   ┆        ┆        ┆              ┆ f64         │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪════════╪════════╪══════════════╪═════════════╡
│ 56215     ┆ null      ┆ 52920     ┆ 2671      ┆ … ┆ 1      ┆ 1      ┆ 1.0          ┆ 1.0         │
│ 2860      ┆ null      ┆ 7982      ┆ 202801    ┆ … ┆ 1      ┆ 1      ┆ 1.0          ┆ 1.0         │
│ 100066360 ┆ null      ┆ 4870      ┆ 215281    ┆ … ┆ 1      ┆ 1      ┆

In [17]:
dict_after_seq_1 = df_to_dict_variants(scores)

In [18]:
# write out results
scores.write_csv(f"{outputs_dir}/3_normalized_qaqc_consensus_scores.csv")

## Calculate normalized scores

In [19]:
# prepare data to calculate normalized scores
normalized_scores = scores.filter(pl.col("condition_name").is_in(scoring_conditions))

normalized_scores = normalized_scores.group_by([
    'ad_orf_id', 'db_orf_id', 'db_mut_id', 'retest_batch'
]).agg(
    (pl.col("adjusted_score").sum() / (len(scoring_conditions)*max_score)).alias("normalized_score")
)

normalized_scores = normalized_scores.with_columns(
    pl.col("retest_batch").str.extract(r"VUSAPWT(\d)B\d+", 1).alias("assay_id")
)
normalized_scores

ad_orf_id,db_orf_id,db_mut_id,retest_batch,normalized_score,assay_id
i64,i64,i64,str,f64,str
14221,14221,218283,"""VUSAPWT1B2""",0.9375,"""1"""
2098,71642,0,"""VUSAPWT1B2""",0.4375,"""1"""
8357,71001,205675,"""VUSAPWT6B1""",0.75,"""6"""
13850,71001,205675,"""VUSAPWT6B1""",0.5625,"""6"""
100066551,7652,1806,"""VUSAPWT6B1""",0.0,"""6"""
…,…,…,…,…,…
53964,2713,200850,"""VUSAPWT1B1""",0.625,"""1"""
100066360,11443,0,"""VUSAPWT1B2""",0.8125,"""1"""
71001,52920,2662,"""VUSAPWT1B1""",1.0,"""1"""


In [20]:
# cast all to same format
normalized_scores = normalized_scores.with_columns(
    pl.col("ad_orf_id").cast(pl.Float64).cast(pl.Int64).alias("ad_orf_id"),
    pl.col("db_orf_id").cast(pl.Float64).cast(pl.Int64).alias("db_orf_id"),
    pl.col("db_mut_id").cast(pl.Float64).cast(pl.Int64).alias("db_mut_id"),
    pl.col("assay_id").cast(pl.String).alias("assay_id"),
)

db_seq = db_seq.with_columns(
    pl.col("db_orf_id").cast(pl.Float64).cast(pl.Int64).alias("db_orf_id"),
    pl.col("db_mut_id").cast(pl.Float64).cast(pl.Int64).alias("db_mut_id"),
    pl.col("seq_db").cast(pl.String).alias("seq_db")
)

ad_seq = ad_seq.with_columns(
    pl.col("ad_orf_id").cast(pl.Float64).cast(pl.Int64).alias("ad_orf_id"),
    pl.col("seq_ad").cast(pl.String).alias("seq_ad"),
    pl.col("assay_id").cast(pl.String).alias("assay_id"),
)

normalized_scores = normalized_scores.join(ad_seq, on=["ad_orf_id", "assay_id"], how="left")
normalized_scores = normalized_scores.join(db_seq, on=["db_orf_id", "db_mut_id"], how="left")


# Filter to only keep interactions where both ad and db are confirmed
normalized_scores = normalized_scores.filter(
    (pl.col("seq_ad") == "1") & (pl.col("seq_db") == "1")
)
normalized_scores

ad_orf_id,db_orf_id,db_mut_id,retest_batch,normalized_score,assay_id,seq_ad,seq_db,db_sequenced,db_sequence_confirmation_class
i64,i64,i64,str,f64,str,str,str,f64,f64
14221,14221,218283,"""VUSAPWT1B2""",0.9375,"""1""","""1""","""1""",1.0,1.0
2098,71642,0,"""VUSAPWT1B2""",0.4375,"""1""","""1""","""1""",null,null
8357,71001,205675,"""VUSAPWT6B1""",0.75,"""6""","""1""","""1""",1.0,1.0
13850,71001,205675,"""VUSAPWT6B1""",0.5625,"""6""","""1""","""1""",1.0,1.0
100066551,7652,1806,"""VUSAPWT6B1""",0.0,"""6""","""1""","""1""",1.0,2.0
…,…,…,…,…,…,…,…,…,…
53964,2713,200850,"""VUSAPWT1B1""",0.625,"""1""","""1""","""1""",1.0,1.0
100066360,11443,0,"""VUSAPWT1B2""",0.8125,"""1""","""1""","""1""",null,null
71001,52920,2662,"""VUSAPWT1B1""",1.0,"""1""","""1""","""1""",1.0,1.0


In [21]:
dict_after_seq_2 = df_to_dict_variants(normalized_scores)

In [22]:
# # map ad orfs ids to symbol

# horfeome_df = pd.read_csv(f"{meta_inputs}/horfeome_81_91_union.tsv", sep="\t")
# orf_to_symbol_dict = horfeome_df.set_index('orf_id')['symbol'].to_dict()

# # add a key for 0 as empty_AD
# orf_to_symbol_dict[0] = 'empty_AD'

# # update values where orfs are mapped to multiple symbols to only retain one previously used in the lab
# single_symbol_mapping_dict = {
#     1063: 'SMN2',
#     13954: 'AC005833.1',
#     55237: 'NBPF19',
#     2658: 'ZNF559-ZNF177',
#     1940: 'RBMY1F',
#     56585: 'CKMT1A',
#     2794: 'HBA1',
#     978: 'MAGEA2B',
#     70851: 'FAM25G',
#     3824: 'CKMT1A',
#     13813: 'CRYAA',
#     66: 'SMN1',
#     55944: 'KRTAP2-4',
#     13597: 'FAM187A',
#     56508: 'PRR20E',
#     100066377: 'TBC1D3G'
#     }


# for k, v in single_symbol_mapping_dict.items():
#     orf_to_symbol_dict[k] = v


# # apply mapping
# normalized_scores = normalized_scores.with_columns(
#     pl.col("ad_orf_id").replace_strict(orf_to_symbol_dict, default=None).alias("ad_symbol")
# )

In [23]:
# map ad_orf_id to ad_symbol
mapping_file = pd.read_csv(f"{inputs_dir}/cp_files/mapping_y2h.csv")
mapping_file.dropna(subset=['ad_orf_id'], inplace=True)
mapping_dict = mapping_file.set_index('ad_orf_id')['ad_symbol'].to_dict()

normalized_scores = normalized_scores.with_columns(
    pl.col("ad_orf_id").replace_strict(mapping_dict, default=None).alias("ad_symbol")
)
normalized_scores

ad_orf_id,db_orf_id,db_mut_id,retest_batch,normalized_score,assay_id,seq_ad,seq_db,db_sequenced,db_sequence_confirmation_class,ad_symbol
i64,i64,i64,str,f64,str,str,str,f64,f64,str
14221,14221,218283,"""VUSAPWT1B2""",0.9375,"""1""","""1""","""1""",1.0,1.0,"""KCTD1"""
2098,71642,0,"""VUSAPWT1B2""",0.4375,"""1""","""1""","""1""",null,null,"""EFHC2"""
8357,71001,205675,"""VUSAPWT6B1""",0.75,"""6""","""1""","""1""",1.0,1.0,"""ERP27"""
13850,71001,205675,"""VUSAPWT6B1""",0.5625,"""6""","""1""","""1""",1.0,1.0,"""CSN2"""
100066551,7652,1806,"""VUSAPWT6B1""",0.0,"""6""","""1""","""1""",1.0,2.0,"""MAPK1"""
…,…,…,…,…,…,…,…,…,…,…
53964,2713,200850,"""VUSAPWT1B1""",0.625,"""1""","""1""","""1""",1.0,1.0,"""NFIX"""
100066360,11443,0,"""VUSAPWT1B2""",0.8125,"""1""","""1""","""1""",null,null,"""KRT27"""
71001,52920,2662,"""VUSAPWT1B1""",1.0,"""1""","""1""","""1""",1.0,1.0,"""UBQLN2"""


In [24]:
# Append metadata and write out file
metadata = pl.read_csv(f"{meta_outputs}/slim_metadata.csv").rename({
    "orf_id": "db_orf_id",
    "mut_id": "db_mut_id"
})

normalized_scores = normalized_scores.join(metadata, on=["db_orf_id", "db_mut_id"], how="left")
normalized_scores

ad_orf_id,db_orf_id,db_mut_id,retest_batch,normalized_score,assay_id,seq_ad,seq_db,db_sequenced,db_sequence_confirmation_class,ad_symbol,symbol,aa_change,nt_change,ensembl_gene_id,collection,clinvar_clnsig_clean,gnomad_af,StarStatus,allele_0
i64,i64,i64,str,f64,str,str,str,f64,f64,str,str,str,str,str,str,str,f64,f64,str
14221,14221,218283,"""VUSAPWT1B2""",0.9375,"""1""","""1""","""1""",1.0,1.0,"""KCTD1""","""KCTD1""","""Gly62Asp""","""185G>A""","""ENSG00000134504""","""CEGS2""","""1_Pathogenic""",null,1.0,"""KCTD1_Gly62Asp"""
2098,71642,0,"""VUSAPWT1B2""",0.4375,"""1""","""1""","""1""",null,null,"""EFHC2""","""CACNB4""",null,null,"""ENSG00000182389""",null,null,null,null,"""CACNB4"""
8357,71001,205675,"""VUSAPWT6B1""",0.75,"""6""","""1""","""1""",1.0,1.0,"""ERP27""","""UBQLN2""","""Pro506Thr""","""1516C>A""","""ENSG00000188021""","""CEGS2""","""1_Pathogenic""",null,1.0,"""UBQLN2_Pro506Thr"""
13850,71001,205675,"""VUSAPWT6B1""",0.5625,"""6""","""1""","""1""",1.0,1.0,"""CSN2""","""UBQLN2""","""Pro506Thr""","""1516C>A""","""ENSG00000188021""","""CEGS2""","""1_Pathogenic""",null,1.0,"""UBQLN2_Pro506Thr"""
100066551,7652,1806,"""VUSAPWT6B1""",0.0,"""6""","""1""","""1""",1.0,2.0,"""MAPK1""","""STXBP1""","""Asp238Glu""","""714C>A""","""ENSG00000136854""","""Edgotyping3""","""4_VUS""",null,1.0,"""STXBP1_Asp238Glu"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
53964,2713,200850,"""VUSAPWT1B1""",0.625,"""1""","""1""","""1""",1.0,1.0,"""NFIX""","""MLH1""","""Ser295Thr""","""884G>C""","""ENSG00000076242""","""CEGS2""","""4_VUS""",null,3.0,"""MLH1_Ser295Thr"""
100066360,11443,0,"""VUSAPWT1B2""",0.8125,"""1""","""1""","""1""",null,null,"""KRT27""","""CCBE1""",null,null,"""ENSG00000183287""",null,null,null,null,"""CCBE1"""
71001,52920,2662,"""VUSAPWT1B1""",1.0,"""1""","""1""","""1""",1.0,1.0,"""UBQLN2""","""LITAF""","""Arg160His""","""479G>A""","""ENSG00000189067""","""Edgotyping3""","""4_VUS""",null,2.0,"""LITAF_Arg160His"""


In [25]:
normalized_scores.write_csv(f"{outputs_dir}/4_normalized_consensus_scores.csv")

# Exploring qa/qc, dropped alleles

In [ ]:
def describe_dict(df_dict):
    for assay_id in df_dict.keys():
        print(f"\tAssay ID: {assay_id}")
        n_db_orfs = len(df_dict[assay_id].keys())
        only_wt_count = 0
        db_list_wt = []
        n_wt = 0
        n_alleles = 0
        db_list_wt_with_var_wt = []
        db_list_wt_with_only_var = []
        ad_list = []
        
        for db_orf_id, db_mut_d in df_dict[assay_id].items():
            # if db_orf_id == 53798:
            #     print('53798')
            #     print(db_mut_d)
            
            if 0 in db_mut_d.keys():
                n_wt += 1
                n_alleles += len(db_mut_d.keys()) - 1
            else:
                n_alleles += len(db_mut_d.keys())
                
            if len(db_mut_d.keys()) == 1 and 0 in db_mut_d.keys():
                only_wt_count += 1
                db_list_wt.append(db_orf_id)
                
            if len(db_mut_d.keys()) > 1 and 0 in db_mut_d.keys():
                db_list_wt_with_var_wt.append(db_orf_id)
                
            if 0 not in db_mut_d.keys():
                db_list_wt_with_only_var.append(db_orf_id)
                
            ad_set = set()
            for db_mut_id, ad_orf_id_set in db_mut_d.items():
                for ad_orf_id in ad_orf_id_set:
                    ad_set.add(ad_orf_id)
                    
            ad_list.extend(list(ad_set))
            
            
        print(f"\t\tNumber of DB ORFs: {n_db_orfs}")        
        print(f"\t\tNumber of DB alleles: {n_alleles}")
        print(f"\t\tNumber of DB ORFs with WT: {n_wt}")
        print(f"\t\tNumber of DB ORFs with only WT: {only_wt_count}, {(only_wt_count/n_db_orfs)*100:.2f}%")
        print(f"\t\tNumber of DB ORFs with WT and alleles: {len(db_list_wt_with_var_wt)}, {(len(db_list_wt_with_var_wt)/n_db_orfs)*100:.2f}%")
        print(f"\t\tNumber of DB ORFs with only alleles: {len(db_list_wt_with_only_var)}, {(len(db_list_wt_with_only_var)/n_db_orfs)*100:.2f}%")
        print(f"\t\tNumber of AD ORFs: {len(ad_list)}")

        print(f"\t\tDB ORFs with only WT: {db_list_wt}")
        print(f"\t\tDB ORFs with WT and alleles: {db_list_wt_with_var_wt}")
        print(f"\t\tDB ORFs with only alleles: {db_list_wt_with_only_var}")
        print()

In [56]:
def compare_dicts(dict1, dict2):
    assay_array = ["VUSAPWT1B1", "VUSAPWT1B2", "VUSAPWT2B1", "VUSAPWT6B1"]
    for assay in assay_array:
        print(f"\tAssay: {assay}")

        lost_alleles = 0
        lost_wt = 0
        lost_db_orfs = 0
        lost_ad_orfs = 0

        total_alleles = 0
        total_wt = 0
        total_db_orfs = 0
        total_ad_orfs = 0

        # Compute totals from dict1
        for db_orf_id, db_mut_id_d in dict1[assay].items():
            total_db_orfs += 1
            if 0 in db_mut_id_d:
                total_wt += 1
                total_alleles += len(db_mut_id_d) - 1  # exclude WT
            else:
                total_alleles += len(db_mut_id_d)
                
            ad_set = set()
                
            ad_set = {ad for db_mut, ad_orf in db_mut_id_d.items() for ad in ad_orf}
            total_ad_orfs += len(ad_set)

        for db_orf_id, db_mut_id_d in dict2[assay].items():
            if db_orf_id in dict1[assay]:
                db_mut_id_d1 = dict1[assay][db_orf_id]
                db_mut_id_d2 = db_mut_id_d

                ad_set_1 = {ad for db_mut, ad_orf in db_mut_id_d1.items() for ad in ad_orf}
                ad_set_2 = {ad for db_mut, ad_orf in db_mut_id_d2.items() for ad in ad_orf}
                lost_ad_orfs += len(ad_set_1 - ad_set_2)

                lost_alleles += sum(1 for db_mut in db_mut_id_d1 if db_mut != 0 and db_mut not in db_mut_id_d2)

                if 0 in db_mut_id_d1 and 0 not in db_mut_id_d2:
                    lost_wt += 1

            else:
                lost_db_orfs += 1
                if 0 in db_mut_id_d:
                    lost_wt += 1
                    lost_alleles += len(db_mut_id_d) - 1
                else:
                    lost_alleles += len(db_mut_id_d)

                lost_ad_orfs += len({ad for db_mut, ad_orf in db_mut_id_d.items() for ad in ad_orf})

        for db_orf_id, db_mut_id_d in dict1[assay].items():
            if db_orf_id not in dict2[assay]:
                lost_db_orfs += 1
                if 0 in db_mut_id_d:
                    lost_wt += 1
                    lost_alleles += len(db_mut_id_d) - 1
                else:
                    lost_alleles += len(db_mut_id_d)
                lost_ad_orfs += len({ad for db_mut, ad_orf in db_mut_id_d.items() for ad in ad_orf})

        def pct(x, total):
            return f"{(x / total * 100):.1f}%" if total else "NA"

        print(f"\t\tLost DB ORFs: {lost_db_orfs} / {total_db_orfs} ({pct(lost_db_orfs, total_db_orfs)})")
        print(f"\t\tLost alleles: {lost_alleles} / {total_alleles} ({pct(lost_alleles, total_alleles)})")
        print(f"\t\tLost WT: {lost_wt} / {total_wt} ({pct(lost_wt, total_wt)})")
        print(f"\t\tLost AD ORFs: {lost_ad_orfs} / {total_ad_orfs} ({pct(lost_ad_orfs, total_ad_orfs)})")
        print()


In [57]:
dicts = [dict_before_qc, dict_after_db_groups, dict_after_db_conditions ,dict_after_mating, dict_after_aa, dict_after_seq_1, dict_after_seq_2]

In [58]:
print("Data filtering through QC/QA:")
print()
print()
print('Before QC:')
print()
describe_dict(dict_before_qc)
print()
print('Step 1: Remove all db groups with the same allele if there was an autoactivation mating failure')
print()
describe_dict(dict_after_db_groups)
print()
print('Differences before QC and step 1:')
print()
compare_dicts(dict_before_qc, dict_after_db_groups)
print()
print('________________________________________________________________________________')
print('Step 2: Remove all db conditions with the same allele if there was an autoactivation mating failure')
print()
describe_dict(dict_after_db_conditions)
print()
print()
print('Differences between step 1 and step 2:')
print()
compare_dicts(dict_after_db_groups, dict_after_db_conditions)
print()
print('________________________________________________________________________________')
print('Step 3: remove mating failures - condition 1 shows no or little yeast growth')
print()
describe_dict(dict_after_mating)
print()
print('Differences between step 2 and step 3:')
print()
compare_dicts(dict_after_db_conditions, dict_after_mating)
print()
print('________________________________________________________________________________')
print('Step 4: Remove autoactivation rows from the main scores df')
print()
describe_dict(dict_after_aa)
print()
print('Differences between step 3 and step 4:')
print()
compare_dicts(dict_after_mating, dict_after_aa)
print()
print('________________________________________________________________________________')
print('Step 5: Filter to only keep interactions where both ad and db are confirmed, before normalizing')
print()
describe_dict(dict_after_seq_1)
print()
print('Differences between step 4 and step 5:')
print()
compare_dicts(dict_after_aa, dict_after_seq_1)
print()
print('________________________________________________________________________________')
print('Step 6: Filter to only keep interactions where both ad and db are confirmed, after normalizing')
print()
describe_dict(dict_after_seq_2)
print()
print('Differences between step 5 and step 6:')
print()
compare_dicts(dict_after_seq_1, dict_after_seq_2)


Data filtering through QC/QA:


Before QC:

	Assay ID: VUSAPWT1B1
		Number of DB ORFs: 23
		Number of DB alleles: 589
		Number of DB ORFs with WT: 23
		Number of DB ORFs with only WT: 0, 0.00%
		Number of DB ORFs with WT and alleles: 23, 100.00%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 104
		DB ORFs with only WT: []
		DB ORFs with WT and alleles: [281, 2651, 2713, 3535, 4723, 4765, 7167, 7215, 7323, 7652, 11805, 12573, 13295, 52920, 54308, 54914, 55246, 56661, 70497, 71001, 71337, 71892, 100016069]
		DB ORFs with only alleles: []

	Assay ID: VUSAPWT1B2
		Number of DB ORFs: 251
		Number of DB alleles: 506
		Number of DB ORFs with WT: 251
		Number of DB ORFs with only WT: 107, 42.63%
		Number of DB ORFs with WT and alleles: 144, 57.37%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 910
		DB ORFs with only WT: [138, 253, 268, 747, 1155, 1284, 1485, 1601, 1884, 2012, 2055, 2380, 2430, 2442, 2448, 2613, 2816, 2818, 2866, 3125, 3192, 3292, 35

In [59]:
compare_dicts(dict_before_qc, dict_after_mating)

	Assay: VUSAPWT1B1
		Lost DB ORFs: 0 / 23 (0.0%)
		Lost alleles: 8 / 589 (1.4%)
		Lost WT: 0 / 23 (0.0%)
		Lost AD ORFs: 0 / 104 (0.0%)

	Assay: VUSAPWT1B2
		Lost DB ORFs: 1 / 251 (0.4%)
		Lost alleles: 7 / 506 (1.4%)
		Lost WT: 1 / 251 (0.4%)
		Lost AD ORFs: 2 / 910 (0.2%)

	Assay: VUSAPWT2B1
		Lost DB ORFs: 4 / 64 (6.2%)
		Lost alleles: 11 / 275 (4.0%)
		Lost WT: 5 / 64 (7.8%)
		Lost AD ORFs: 26 / 256 (10.2%)

	Assay: VUSAPWT6B1
		Lost DB ORFs: 0 / 83 (0.0%)
		Lost alleles: 10 / 361 (2.8%)
		Lost WT: 0 / 83 (0.0%)
		Lost AD ORFs: 4 / 322 (1.2%)



In [60]:
compare_dicts(dict_after_mating, dict_after_aa)

	Assay: VUSAPWT1B1
		Lost DB ORFs: 0 / 23 (0.0%)
		Lost alleles: 0 / 581 (0.0%)
		Lost WT: 0 / 23 (0.0%)
		Lost AD ORFs: 0 / 104 (0.0%)

	Assay: VUSAPWT1B2
		Lost DB ORFs: 0 / 250 (0.0%)
		Lost alleles: 10 / 499 (2.0%)
		Lost WT: 0 / 250 (0.0%)
		Lost AD ORFs: 0 / 908 (0.0%)

	Assay: VUSAPWT2B1
		Lost DB ORFs: 1 / 60 (1.7%)
		Lost alleles: 1 / 264 (0.4%)
		Lost WT: 1 / 59 (1.7%)
		Lost AD ORFs: 2 / 230 (0.9%)

	Assay: VUSAPWT6B1
		Lost DB ORFs: 0 / 83 (0.0%)
		Lost alleles: 0 / 351 (0.0%)
		Lost WT: 0 / 83 (0.0%)
		Lost AD ORFs: 0 / 318 (0.0%)



In [61]:
compare_dicts(dict_after_aa, dict_after_seq_1)

	Assay: VUSAPWT1B1
		Lost DB ORFs: 2 / 23 (8.7%)
		Lost alleles: 182 / 581 (31.3%)
		Lost WT: 2 / 23 (8.7%)
		Lost AD ORFs: 25 / 104 (24.0%)

	Assay: VUSAPWT1B2
		Lost DB ORFs: 32 / 250 (12.8%)
		Lost alleles: 167 / 489 (34.2%)
		Lost WT: 36 / 250 (14.4%)
		Lost AD ORFs: 166 / 908 (18.3%)

	Assay: VUSAPWT2B1
		Lost DB ORFs: 7 / 59 (11.9%)
		Lost alleles: 78 / 263 (29.7%)
		Lost WT: 7 / 58 (12.1%)
		Lost AD ORFs: 49 / 228 (21.5%)

	Assay: VUSAPWT6B1
		Lost DB ORFs: 6 / 83 (7.2%)
		Lost alleles: 102 / 351 (29.1%)
		Lost WT: 7 / 83 (8.4%)
		Lost AD ORFs: 20 / 318 (6.3%)



In [62]:
describe_dict(dict_before_qc)

	Assay ID: VUSAPWT1B1
		Number of DB ORFs: 23
		Number of DB alleles: 589
		Number of DB ORFs with WT: 23
		Number of DB ORFs with only WT: 0, 0.00%
		Number of DB ORFs with WT and alleles: 23, 100.00%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 104
		DB ORFs with only WT: []
		DB ORFs with WT and alleles: [281, 2651, 2713, 3535, 4723, 4765, 7167, 7215, 7323, 7652, 11805, 12573, 13295, 52920, 54308, 54914, 55246, 56661, 70497, 71001, 71337, 71892, 100016069]
		DB ORFs with only alleles: []

	Assay ID: VUSAPWT1B2
		Number of DB ORFs: 251
		Number of DB alleles: 506
		Number of DB ORFs with WT: 251
		Number of DB ORFs with only WT: 107, 42.63%
		Number of DB ORFs with WT and alleles: 144, 57.37%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 910
		DB ORFs with only WT: [138, 253, 268, 747, 1155, 1284, 1485, 1601, 1884, 2012, 2055, 2380, 2430, 2442, 2448, 2613, 2816, 2818, 2866, 3125, 3192, 3292, 3594, 3643, 3668, 3670, 3693, 3699, 3844, 3878

In [63]:
describe_dict(dict_after_mating)

	Assay ID: VUSAPWT1B1
		Number of DB ORFs: 23
		Number of DB alleles: 581
		Number of DB ORFs with WT: 23
		Number of DB ORFs with only WT: 0, 0.00%
		Number of DB ORFs with WT and alleles: 23, 100.00%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 104
		DB ORFs with only WT: []
		DB ORFs with WT and alleles: [281, 2651, 2713, 3535, 4723, 4765, 7167, 7215, 7323, 7652, 11805, 12573, 13295, 52920, 54308, 54914, 55246, 56661, 70497, 71001, 71337, 71892, 100016069]
		DB ORFs with only alleles: []

	Assay ID: VUSAPWT1B2
		Number of DB ORFs: 250
		Number of DB alleles: 499
		Number of DB ORFs with WT: 250
		Number of DB ORFs with only WT: 107, 42.80%
		Number of DB ORFs with WT and alleles: 143, 57.20%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 908
		DB ORFs with only WT: [138, 253, 268, 747, 1155, 1284, 1485, 1601, 1884, 2012, 2055, 2380, 2430, 2442, 2448, 2613, 2816, 2818, 2866, 3125, 3192, 3292, 3594, 3643, 3668, 3670, 3693, 3699, 3844, 3878

In [64]:
describe_dict(dict_after_aa)

	Assay ID: VUSAPWT1B1
		Number of DB ORFs: 23
		Number of DB alleles: 581
		Number of DB ORFs with WT: 23
		Number of DB ORFs with only WT: 0, 0.00%
		Number of DB ORFs with WT and alleles: 23, 100.00%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 104
		DB ORFs with only WT: []
		DB ORFs with WT and alleles: [281, 2651, 2713, 3535, 4723, 4765, 7167, 7215, 7323, 7652, 11805, 12573, 13295, 52920, 54308, 54914, 55246, 56661, 70497, 71001, 71337, 71892, 100016069]
		DB ORFs with only alleles: []

	Assay ID: VUSAPWT1B2
		Number of DB ORFs: 250
		Number of DB alleles: 489
		Number of DB ORFs with WT: 250
		Number of DB ORFs with only WT: 108, 43.20%
		Number of DB ORFs with WT and alleles: 142, 56.80%
		Number of DB ORFs with only alleles: 0, 0.00%
		Number of AD ORFs: 908
		DB ORFs with only WT: [138, 253, 268, 747, 1155, 1284, 1485, 1601, 1884, 2012, 2055, 2380, 2430, 2442, 2448, 2613, 2816, 2818, 2866, 3125, 3192, 3292, 3594, 3643, 3668, 3670, 3693, 3699, 3844, 3878